# Optuna FFDCN 모델

In [1]:
import time
import argparse
import json
import pandas as pd
import tqdm
import argparse
import warnings
import joblib
from src import seed_everything

from src.data import context_data_load, context_data_split, context_data_loader

from src import FFDCNModel

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings(action='ignore')

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
parser = argparse.ArgumentParser()
args = argparse.Namespace()
with open('config.json','rt') as f:
    args.__dict__.update(json.load(f))

In [3]:
seed_everything(42)

In [4]:
ffmdataset = context_data_load(args)

In [5]:
def objective(trial):
    seed_everything(args.SEED)
    args.BATCH_SIZE = trial.suggest_categorical('BATCH_SIZE',[256, 512, 1024])
    args.EPOCHS = 1 #trial.suggest_int('EPOCH',5,10)
    args.LR = trial.suggest_loguniform('LR',0.001,0.01)
    args.WEIGHT_DECAY = trial.suggest_loguniform('WEIGHT_DECAY',1e-07,5e-06)
    args.FFM_EMBED_DIM = trial.suggest_int('FFM_EMBED_DIM', 3, 32)
    args.DCN_EMBED_DIM = trial.suggest_int('DCN_EMBED_DIM', 1, 16)
    DCN_MLP_DIM_LAYERS = trial.suggest_int('DCN_MLP_DIM_LAYERS',1,3)
    args.DCN_MLP_DIMS = [trial.suggest_int('DCN_MLP_DIM_NUM',1,16)]*DCN_MLP_DIM_LAYERS
    args.DCN_DROPOUT = trial.suggest_categorical("DCN_DROPOUT",[0.2,0.25,0.3])
    args.DCN_NUM_LAYERS = trial.suggest_int('DCN_NUM_LAYERS',1 , 4)
    # args.USER_N_D = trial.suggest_int('USER_N_D',0,3)
    # args.USER_F_D = trial.suggest_int('USER_N_F',3,6)
    # args.ISBN_N_D = trial.suggest_categorical('ISBN_N_D',[12,14,16,18,20,22])
    # args.ISBN_N_F = trial.suggest_int('ISBN_N_F',28,32)
    # ffmdataset = context_data_load(args)
    dataffm = context_data_split(args,ffmdataset)
    dataffm = context_data_loader(args,dataffm)
    model = FFDCNModel(args,dataffm)
    model.train()
    log_score = model.predict_train()
    
    return log_score

In [6]:
sampler = optuna.samplers.TPESampler(seed=49)
study = optuna.create_study(
    study_name = 'FFDCN_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)
study.optimize(objective, n_trials=200)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-11-08 04:05:38,171] A new study created in memory with name: FFDCN_parameter_opt


42


  0%|          | 0/60 [00:00<?, ?it/s]

epoch: 0 validation: rmse: 2.204603349985907


100%|██████████| 60/60 [00:00<00:00, 78.28it/s]
[I 2022-11-08 04:05:53,907] Trial 0 finished with value: 2.204603349985907 and parameters: {'BATCH_SIZE': 1024, 'LR': 0.0077911835931194886, 'WEIGHT_DECAY': 1.4483273098046678e-06, 'FFM_EMBED_DIM': 20, 'DCN_EMBED_DIM': 9, 'DCN_MLP_DIM_LAYERS': 1, 'DCN_MLP_DIM_NUM': 13, 'DCN_DROPOUT': 0.2, 'DCN_NUM_LAYERS': 2}. Best is trial 0 with value: 2.204603349985907.


...

42


  0%|          | 0/240 [00:00<?, ?it/s]

epoch: 0 validation: rmse: 2.167270675153166


100%|██████████| 240/240 [00:01<00:00, 217.89it/s]
[I 2022-11-08 05:32:40,302] Trial 199 finished with value: 2.167270675153166 and parameters: {'BATCH_SIZE': 256, 'LR': 0.008526869867156435, 'WEIGHT_DECAY': 2.566910559963766e-07, 'FFM_EMBED_DIM': 14, 'DCN_EMBED_DIM': 8, 'DCN_MLP_DIM_LAYERS': 1, 'DCN_MLP_DIM_NUM': 7, 'DCN_DROPOUT': 0.2, 'DCN_NUM_LAYERS': 2}. Best is trial 192 with value: 2.159267557051403.


Best Score: 2.159267557051403
Best trial {'BATCH_SIZE': 256, 'LR': 0.008076100444004818, 'WEIGHT_DECAY': 3.2164837228677867e-07, 'FFM_EMBED_DIM': 14, 'DCN_EMBED_DIM': 8, 'DCN_MLP_DIM_LAYERS': 1, 'DCN_MLP_DIM_NUM': 8, 'DCN_DROPOUT': 0.2, 'DCN_NUM_LAYERS': 2}


In [7]:
study.get_trials()[-1].params

{'BATCH_SIZE': 256,
 'LR': 0.008526869867156435,
 'WEIGHT_DECAY': 2.566910559963766e-07,
 'FFM_EMBED_DIM': 14,
 'DCN_EMBED_DIM': 8,
 'DCN_MLP_DIM_LAYERS': 1,
 'DCN_MLP_DIM_NUM': 7,
 'DCN_DROPOUT': 0.2,
 'DCN_NUM_LAYERS': 2}

In [8]:
joblib.dump(study,'./valid/studysave1003.pkl')

FileNotFoundError: [Errno 2] No such file or directory: './valid/studysave1003.pkl'

In [ ]:
jl = joblib.load('./valid/studysave1003.pkl')

In [ ]:
jl.best_params

{'BATCH_SIZE': 858,
 'LR': 0.004313538772296255,
 'WEIGHT_DECAY': 2.64921217501965e-07,
 'FFM_EMBED_DIM': 25,
 'DCN_EMBED_DIM': 6,
 'DCN_MLP_DIM_LAYERS': 3,
 'DCN_MLP_DIM_NUM': 1,
 'DCN_DROPOUT': 0.25,
 'DCN_NUM_LAYERS': 3}

## K-Fold for FFDCN

In [9]:
import time
import argparse
import json
import pandas as pd
import numpy as np
import tqdm
import argparse
import warnings
import joblib
from src import seed_everything

from src.data import context_data_load, context_data_split, context_data_loader

from src import FFDCNModel

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings(action='ignore')

In [10]:
parser = argparse.ArgumentParser()
args = argparse.Namespace()
with open('config.json','rt') as f:
    args.__dict__.update(json.load(f))

In [11]:
seed_everything(42)

In [12]:
data = context_data_load(args)

In [13]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(data['train'].drop(['rating'], axis=1), data['train']['rating']):
    folds.append((train_idx, valid_idx))

In [14]:
ffdcn_predicts = []
for fold in range(5):
    seed_everything(42)
    print('='*15,fold+1,'='*15)
    train_idx, valid_idx = folds[fold]
    # Fold i Data Split
    X_train = data['train'].drop(['rating'], axis=1).iloc[train_idx]
    X_valid = data['train'].drop(['rating'], axis=1).iloc[valid_idx]
    y_train = data['train']['rating'][train_idx]
    y_valid = data['train']['rating'][valid_idx]
    # Create Fold i Dataloader
    fold_data = {
            'X_train':X_train,
            'X_valid':X_valid,
            'y_train':y_train,
            'y_valid':y_valid,
            'test':data['test'],
            'field_dims':data['field_dims'],
            'sub':data['sub'],
            'idx2user':data['idx2user'],
            'idx2isbn':data['idx2isbn']
    }
    fold_data = context_data_loader(args,fold_data)
    # Create Fold i FFDCN Model and train
    print(f'--------------- {args.MODEL} TRAINING ---------------')
    model = FFDCNModel(args,fold_data)
    model.train()
    log_score = model.predict_train()

    # Fold i Model's Predict Test data
    print(f'--------------- {args.MODEL} PREDICT ---------------')
    predicts = model.predict(fold_data['test_dataloader'])
    ffdcn_predicts.append(predicts)



=============== 1 ===============
--------------- FFDCN TRAINING ---------------


  0%|          | 0/240 [00:00<?, ?it/s]

epoch: 0 validation: rmse: 2.1640109036934394


  0%|          | 0/300 [00:00<?, ?it/s]

--------------- FFDCN PREDICT ---------------


...

  0%|          | 0/240 [00:00<?, ?it/s]

epoch: 0 validation: rmse: 2.1739715771745107


  0%|          | 0/300 [00:00<?, ?it/s]

--------------- FFDCN PREDICT ---------------


100%|██████████| 300/300 [00:01<00:00, 256.63it/s]


In [17]:
# Fold i Save Predicted test data
print(f'--------------- SAVE {args.MODEL} PREDICT ---------------')
submission = pd.read_csv(args.DATA_PATH + 'sample_submission.csv')
for fold_predict in ffdcn_predicts:
    submission['rating'] += np.array(fold_predict) / 5
submission.loc[submission['rating']>10,'rating'] = 10.0
now = time.localtime()
now_date = time.strftime('%Y%m%d', now)
now_hour = time.strftime('%X', now)
save_time = now_date + '_' + now_hour.replace(':', '')
#submission.to_csv('submit/5fold_{}_{}.csv'.format(save_time, args.MODEL), index=False)
submission.to_csv('submit/FFDCN.csv', index=False)

--------------- SAVE FFDCN PREDICT ---------------
